In [1]:
# robust_feature_extractor.py
import os, pathlib, warnings
import numpy as np
import pandas as pd
import librosa
from scipy.stats import skew, kurtosis
from tqdm import tqdm

# ----------------------------- Genel ayarlar ------------------------------
DATA_PATH    = pathlib.Path('/Users/canpasa/Desktop/Tezle İlgili/TEZ SAMPLES DENEME 1703')
SR           = 22_050
N_FFT        = 1024
HOP_LENGTH   = 128
N_MEL        = 128
N_MFCC       = 20
ROLL_PERCENT = 0.85
EPS          = 1e-10

# ----------------------------- Öznitelikler -------------------------------
def log_attack_features(env, sr, hop):
    if env.max() < EPS:
        return 0.0, 0.0
    peak_idx    = np.argmax(env)
    env_norm    = env / env.max() + EPS
    try:
        t10 = np.where(env_norm >= 0.10)[0][0]
        t90 = np.where(env_norm >= 0.90)[0][0]
    except IndexError:
        return (peak_idx * hop) / sr, 0.0
    attack_time = (t90 - t10) * hop / sr
    if attack_time < EPS:
        return attack_time, 0.0
    env_db      = librosa.amplitude_to_db(env_norm, ref=1.0)
    slope       = (env_db[t90] - env_db[t10]) / attack_time
    return attack_time, slope

def hpss_energy_ratio(y):
    y_h, y_p = librosa.effects.hpss(y)
    return np.sum(y_p**2) / (np.sum(y_h**2) + EPS)

def extract_features(signal, sr):
    if len(signal) < N_FFT:
        signal = np.pad(signal, (0, N_FFT - len(signal)))
    S   = np.abs(librosa.stft(signal, n_fft=N_FFT, hop_length=HOP_LENGTH))
    mel = librosa.feature.melspectrogram(S=S, sr=sr, n_mels=N_MEL)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=N_MFCC)

    # --- RMS -----------------------------------------------------------------
    rms = librosa.feature.rms(
    S=S,
    frame_length=N_FFT,        #  <<< 1024
    hop_length=HOP_LENGTH
    )[0]

# --- ZCR -----------------------------------------------------------------
    zcr = librosa.feature.zero_crossing_rate(
    signal,
    frame_length=N_FFT,        #  <<< 1024
    hop_length=HOP_LENGTH
    )[0]

    centroid = librosa.feature.spectral_centroid(S=S, sr=sr)[0]
    bandwidth= librosa.feature.spectral_bandwidth(S=S, sr=sr)[0]
    rolloff  = librosa.feature.spectral_rolloff(S=S, sr=sr,
                                                roll_percent=ROLL_PERCENT)[0]
    flatness = librosa.feature.spectral_flatness(S=S)[0]
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    flux     = librosa.onset.onset_strength(S=S, sr=sr,
                                            hop_length=HOP_LENGTH)
    onset_env= librosa.onset.onset_strength(y=signal, sr=sr,
                                            n_fft=N_FFT,
                                            hop_length=HOP_LENGTH)
    atk_time, atk_slope = log_attack_features(onset_env, sr, HOP_LENGTH)

    feats = {
        **{f"mfcc{i+1:02d}": mfcc[i].mean() for i in range(N_MFCC)},
        "rms_mean": np.mean(rms),
        "rms_std":  np.std(rms),
        "zcr_mean": np.mean(zcr),
        "centroid_mean": np.mean(centroid),
        "bandwidth_mean": np.mean(bandwidth),
        "rolloff_mean": np.mean(rolloff),
        "flatness_mean": np.mean(flatness),
        "flux_mean": np.mean(flux),
        **{f"contrast_b{b+1}": contrast[b].mean() for b in range(contrast.shape[0])},
        "onset_mean":  onset_env.mean(),
        "onset_std":   onset_env.std(),
        "onset_max":   onset_env.max(),
        "onset_sum":   onset_env.sum(),
        "attack_time": atk_time,
        "attack_slope": atk_slope,
        "hpi_ratio":   hpss_energy_ratio(signal),
    }
    return feats

# --------------------------- Dizin taraması -------------------------------
records, bad_files = [], []
warnings.filterwarnings("ignore")

for dirpath, _, filenames in os.walk(DATA_PATH):
    label = pathlib.Path(dirpath).name
    wavs  = [f for f in filenames if f.lower().endswith(".wav")]

    for wav in tqdm(wavs, desc=label, leave=False):
        file_path = pathlib.Path(dirpath) / wav
        try:
            signal, _ = librosa.load(file_path, sr=SR)
        except Exception as e:
            print(f"[SKIP] {file_path}: {e}")
            bad_files.append(file_path)
            continue

        feat_dict = extract_features(signal, SR)
        feat_dict.update({"label": label, "filename": wav})
        records.append(feat_dict)

# ----------------------- Sonuç DataFrame & rapor ---------------------------
df = pd.DataFrame.from_records(records)
print(f"\nToplam özellik çıkarılan dosya: {len(df)}")
print(f"Atlanan sorunlu dosya      : {len(bad_files)}")
# df.to_csv("one_shot_features_clean.csv", index=False)




Toplam özellik çıkarılan dosya: 6435
Atlanan sorunlu dosya      : 0


In [2]:
df

,mfcc01,mfcc02,mfcc03,mfcc04,mfcc05,mfcc06,mfcc07,mfcc08,mfcc09,mfcc10,...,contrast_b7,onset_mean,onset_std,onset_max,onset_sum,attack_time,attack_slope,hpi_ratio,label,filename
0,-375.807861,40.318409,2.222316,25.521338,17.871218,14.569949,6.145129,8.773415,7.858817,10.529112,...,42.412579,0.337117,0.525832,3.022758,28.654934,0.267029,22.350317,0.705611,Kick,053.vubeatz - Kick-19.wav
1,-399.842896,105.145592,5.199309,10.956247,24.178602,15.623933,7.570135,11.298717,8.760970,10.446151,...,33.525089,0.210629,0.190053,0.680425,9.688943,0.185760,101.448700,1.952946,Kick,061.vubeatz - Kick-27.wav
2,-442.545380,101.096001,42.732346,34.081097,21.172230,19.151922,13.419593,12.518649,8.550889,11.492721,...,30.396937,0.087409,0.201809,1.219732,3.321545,0.000000,0.000000,1.492114,Kick,Oversampled_SH2_kick_trap_heat.wav
3,-517.262390,59.168682,37.639400,28.265085,18.350937,15.631050,10.798821,11.485106,8.637713,10.242297,...,36.001919,0.031550,0.100115,0.912067,3.028791,0.000000,0.000000,1.699616,Kick,WLUB2 Kick 12.wav
4,-386.069611,169.830566,26.742741,34.934872,16.483923,17.485287,10.294847,10.084108,5.280774,6.466370,...,23.650770,0.083986,0.172245,0.901509,2.267620,0.000000,0.000000,0.847363,Kick,M - SATYR KICK.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,-437.953339,17.951803,1.051005,10.281075,5.732579,7.572130,5.026505,6.844545,2.628994,4.305309,...,43.381839,0.057343,0.216664,1.976624,4.988809,0.000000,0.000000,34555.600919,Rims,WDRIM.WAV
6431,-329.378601,49.843285,-11.793707,3.316524,-7.644785,-2.481364,-10.530240,-6.370984,-7.775508,0.357688,...,25.417838,0.069979,0.202533,0.709305,0.769770,0.000000,0.000000,0.527696,Rims,TR808 RS Rim Shot 05.wav
6432,-780.793579,23.670181,10.198466,11.962135,9.164957,9.230130,7.387212,7.402624,6.249685,7.915323,...,18.400467,0.042422,0.221765,1.800706,3.139211,0.000000,0.000000,266998.135447,Rims,Perc - Raze [A].wav
6433,-246.219284,65.072815,-62.704678,-15.666989,-25.249407,-5.989382,3.348836,4.341005,6.100430,10.192139,...,35.353649,0.125798,0.299213,1.417920,2.767566,0.000000,0.000000,9.773216,Rims,Oversampled_SH2_rim_trap_excursion.wav


In [3]:
df.to_csv("one_shot_features_clean_V4.csv", index=False)